***Foundations of Investment Analysis***, Bates, Boyer, and Fletcher

# Example Chapter 5: Accessing Data from the Internet
In this example we tap into the yfinance library to get monthly stock returns and Ken French's website to access data on short-term US t-bill returns. To make the process of gathering data from these sources as easy as possible, we package most of the code in easy-to-use functions in the library `simple_finance.py`.  We then use these data to estimate the annualized expected return, volatility, and Sharpe ratio for the Russell 3000 index (^RUT) and Ford Motor Company (F).  Try changing the tickers and date ranges to pull data for different companies over different time periods.


 ### Imports and Setup
 We first load in a function file from the GitHub repository using the `curl` command. the `-O` following this command tells curl to save the file `simple_finance.py` in the current working directory. We then import the functions from the file and import other necessary libraries.

In [14]:
# Load in simple_finance.py from the GitHub repository
!curl -O https://raw.githubusercontent.com/boyerb/Investments/master/functions/simple_finance.py
!ls
import simple_finance as sf
import inspect
# import other necessary libraries
import numpy as np
import yfinance as yf
import pandas as pd
print(inspect.getsource(sf))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6294  100  6294    0     0  65838      0 --:--:-- --:--:-- --:--:-- 66252
__pycache__  sample_data  simple_finance.py
import zipfile
from io import BytesIO
import pandas as pd
import yfinance as yf
import requests
import io
import ssl
import urllib.request
import certifi
import urllib.request
import os
from requests.adapters import HTTPAdapter
from urllib3.poolmanager import PoolManager
import urllib3
import statsmodels.api as sm
"""
This functions downloads and processes the Fama-French 5-Factor data from the Dartmouth website using the 'requests' library. 
"""
def get_ff5():
    url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_CSV.zip"

    # Read the zipped CSV file from the URL
    context = ssl.create_default_context(cafile=certifi.where())
    with urllib.request.urlopen

### Access Monthly Returns
In this block of code we access monthly returns for the Russell 3000 index (^RUA) and Ford Motor Company (F). The key function is `get_monthly_returns`.

`get_monthly returns` has the following inputs:
- **ticker** = list of desired tickers: `['ticker_1', 'ticker_2', ..., 'ticker_N']
- **begin** = beginning month in format yyyy-mm-dd
- **end** = ending month in format yyyy-mm-dd
- **tbill_return** = True or False. If True, the return on short-term tbills will be included in the DataFrame. If False, the return on short-term tbills will not be included.

The output is a DataFrame with monthly returns for each ticker, and the return on short-term tbills if `tbill_return=True`.

We then calculate the annualized average return and volatility.

In [16]:
start_date='1990-11-15'
end_date='2023-12-31'
adjusted_start_date = pd.to_datetime(start_date) - pd.DateOffset(months=1)
ticker = ['^RUA', 'F']
data = yf.download(ticker, start=adjusted_start_date, end=end_date, interval="1d")

[*********************100%***********************]  2 of 2 completed


In [17]:
print(data)


Price           Close                    High                     Low  \
Ticker              F         ^RUA          F         ^RUA          F   
Date                                                                    
1990-10-15   1.678722   165.009995   1.685332   165.610001   1.612631   
1990-10-16   1.632459   162.750000   1.711768   165.479996   1.619240   
1990-10-17   1.619239   162.750000   1.665504   163.979996   1.619239   
1990-10-18   1.619239   166.399994   1.652285   166.399994   1.606021   
1990-10-19   1.658895   169.679993   1.658895   169.690002   1.599413   
...               ...          ...        ...          ...        ...   
2023-12-22  11.542161  2741.199951  11.644965  2751.260010  11.476739   
2023-12-26  11.635618  2754.719971  11.682348  2760.330078  11.560851   
2023-12-27  11.579545  2759.219971  11.635620  2761.669922  11.430010   
2023-12-28  11.532816  2759.620117  11.616929  2765.300049  11.476740   
2023-12-29  11.392626  2748.209961  11.579544  2761

In [8]:
# Get monthly returns for a given list of stock tickers using the yfinance API
ticker = ['^RUA', 'F']
# Use tbill_return=True to get the return on short-term tbills, or False to exclude tbill returns
dat = sf.get_monthly_returns(ticker, '1990-01-01', '2023-12-31', tbill_return=False)


[*********************100%***********************]  1 of 1 completed


KeyError: 'Adj Close'

In [ ]:
# Get monthly returns for a given list of stock tickers using the yfinance API
ticker = ['^RUA', 'F']
# Use tbill_return=True to get the return on short-term tbills, or False to exclude tbill returns
dat = sf.get_monthly_returns(ticker, '1990-01-01', '2023-12-31', tbill_return=False)

# Calculate the annualized average return and volatility
A_Avg = dat.mean() * 12
A_Vol = dat.std() * np.sqrt(12)
print('Anualized Average Return')
print(A_Avg)
print()
print('Annualized Volatility')
print(A_Vol)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Anualized Average Return
^RUA    0.089472
F       0.127640
dtype: float64

Annualized Volatility
^RUA    0.152483
F       0.431616
dtype: float64


### Calculate Sharpe Ratios
In this block od code we access data for the Russell 3000 index and Ford Motor Company **ticker = ['^RUA', 'F']**. We also retrieve data on short-term t-bill returns. Using these data we calculate the Sharpe ratios for the Russell 3000 and Ford.

In [ ]:
# Download data along with return on short-term tbills from Ken French's website
dat = sf.get_monthly_returns(ticker, '1990-01-01', '2023-12-31', tbill_return=True)
# Calculate excess returns
dat['exret0'] = dat[ticker[0]] - dat['RF']
dat['exret1'] = dat[ticker[1]] - dat['RF']
print(dat.head())

# Calculate and print Sharpe ratios
# Calculate the Shapre ratio for the first ticker in the list
A_AVG_exret0 = dat['exret0'].mean() * 12
A_Vol_exret0 = dat[ticker[0]].std() * np.sqrt(12)
Sharpe0 = A_AVG_exret0/A_Vol_exret0

# Calculate the Shapre ratio for the second ticker in the list
A_AVG_exret1 = dat['exret1'].mean() * 12
A_Vol_exret1 = dat[ticker[1]].std() * np.sqrt(12)
Sharpe1 = A_AVG_exret1/A_Vol_exret1

print('Sharpe Ratios')
print(ticker[0], Sharpe0)
print(ticker[1], Sharpe1)